In [14]:
# Data handling
import numpy as np
import pandas as pd

In [11]:
"""
Citation:
# P. Cortez and A. Morais. A Data Mining Approach to Predict Forest Fires using Meteorological Data.
# In J. Neves, M. F. Santos and J. Machado Eds., New Trends in Artificial Intelligence,
# Proceedings of the 13th EPIA 2007 - Portuguese Conference on Artificial Intelligence, December,
Guimaraes, Portugal, pp. 512-523, 2007. APPIA, ISBN-13 978-989-95618-0-9.
"""
df = pd.read_csv("forestfires.csv")

In [17]:
# Cleaning data for model by translating to booleans
feats = ['month','day']
data = pd.get_dummies(df,columns=feats,drop_first=True)

In [103]:
# Splitting data for training
from sklearn.model_selection import train_test_split

# Remove area field (this is the expected evaluative output)
X = data.drop(['area'],axis=1).values
y = data['area'].values

# 80% of data for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [104]:
# Scaling data to fit model
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [105]:
# Machine learning with keras using TensorFlow
import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

"""
Hidden layers are calculated as:
        (input_node_count + output_node_count) / 2 = 7
"""

# Input layer
model.add(Dense(7, kernel_initializer = "uniform",activation = "relu", input_dim=27))

# Output layer
model.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

In [106]:
# Gradient descent
model.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

# Fitting model to data
model.fit(X_train, y_train, batch_size = 10, epochs = 1)

42/42 [==============================] - 0s 419us/step - loss: 0.4850 - accuracy: 0.0604


In [98]:
y_pred = model.predict(X_test)

4/4 [==============================] - 0s 833us/step


In [107]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Load and preprocess your dataset
# X: Features, y: Target variable (amount of area burned)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a sequential neural network model
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),  # Input layer
    keras.layers.Dense(128, activation='relu'),  # Hidden layer with ReLU activation
    keras.layers.Dense(64, activation='relu'),   # Another hidden layer
    keras.layers.Dense(1)  # Output layer (1 neuron for regression)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Mean Absolute Error on Test Data: {mae}")

# Make predictions
predictions = model.predict(X_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).